In [1]:
! pip install opendatasets


In [1]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Your Kaggle Key:Your Kaggle Key:Your Kaggle Key:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images


In [ ]:
# Cell 1: Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
# Cell 2: Define image preprocessing functions
def load_image(image_path):
    """Step 1: Load image - Read the image file"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return img

# Cell 5: More preprocessing functions
def resize_image(image):
    """Step 2: Resize - Convert to 224x224 pixels"""
    return cv2.resize(image, (224, 224))

def normalize_image(image):
    """Step 3: Normalize - Scale pixel values to 0-1"""
    return image / 255.0

def convert_to_tensor(image):
    """Step 4: Convert to Tensor - Prepare image for training"""
    return tf.convert_to_tensor(image, dtype=tf.float32)

# Cell 6: Full preprocessing pipeline
# Full preprocessing pipeline for a single image
def preprocess_image(image_path):
    img = load_image(image_path)  # Step 1
    img = resize_image(img)       # Step 2
    img = normalize_image(img)    # Step 3
    img_tensor = convert_to_tensor(img)  # Step 4
    return img_tensor

In [ ]:
# Define directory paths to match your structure
data_root = "/content/deepfake-and-real-images"  # Adjust this if needed
train_dir = os.path.join(data_root, "Dataset/Train")
val_dir = os.path.join(data_root, "Dataset/Validation")
test_dir = os.path.join(data_root, "Dataset/Test")

# Verify directories exist
print(f"Train directory exists: {os.path.exists(train_dir)}")
print(f"Validation directory exists: {os.path.exists(val_dir)}")
print(f"Test directory exists: {os.path.exists(test_dir)}")



In [ ]:
# Create data generators incorporating the preprocessing steps
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Step 3: Normalize
    rotation_range=20,  # Step 5: Data Augmentation
    width_shift_range=0.2,  # Step 5: Data Augmentation
    height_shift_range=0.2,  # Step 5: Data Augmentation
    shear_range=0.2,  # Step 5: Data Augmentation
    zoom_range=0.2,  # Step 5: Data Augmentation
    horizontal_flip=True  # Step 5: Data Augmentation
)

# Only normalize for validation and test (no augmentation needed)
val_datagen = ImageDataGenerator(rescale=1./255)  # Step 3: Normalize
test_datagen = ImageDataGenerator(rescale=1./255)  # Step 3: Normalize
batch_size = 32

In [ ]:
# Create generators
batch_size = 32

# Validation generator
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # Step 2: Resize
    batch_size=batch_size,
    class_mode='binary',
    classes=['Real', 'Fake']  # Changed to match your capitalization
)

# Modify your training data generator to use a subset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.8  # This will use only 20% of your data
)

# Then in your train_generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    classes=['Real', 'Fake'],
    subset='training'  # This will use the training subset
)

In [ ]:
plt.figure(figsize=(15, 8))
sample_batch = next(train_generator)
for i in range(min(9, batch_size)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_batch[0][i])
    plt.title(f"Label: {'Real' if sample_batch[1][i] == 0 else 'Fake'}")
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
def create_model():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification: real vs fake
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Create the model
model = create_model()
print(model.summary())

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3),
    tf.keras.callbacks.ModelCheckpoint(
        'deepfake_detector_model.h5', 
        save_best_only=True, 
        monitor='val_accuracy'
    )
]

In [ ]:
# Cell 13: Set epochs
epochs = 5

In [ ]:
# Train the model using the generators
history = model.fit(
    train_generator,
    steps_per_epoch=200,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks
)


In [ ]:
# Cell 15: Plot training accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Function to make predictions on new images following the preprocessing steps
def predict_image(image_path, model):
    img = load_image(image_path)  # Step 1: Load image
    img = resize_image(img)       # Step 2: Resize
    img = normalize_image(img)    # Step 3: Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension for prediction
    # Step 4: Convert to tensor happens implicitly when passing to model.predict
    
    prediction = model.predict(img)[0][0]
    label = "Real" if prediction < 0.5 else "Fake"  # Adjust based on your class mapping
    confidence = 1 - prediction if prediction < 0.5 else prediction
    
    return label, confidence

In [ ]:
# Function to visualize predictions on multiple test images
def visualize_predictions(model, test_generator, num_images=10):
    # Get a batch of test images
    test_batch = next(test_generator)
    images = test_batch[0][:num_images]
    true_labels = test_batch[1][:num_images]
    
    # Make predictions
    predictions = model.predict(images)
    pred_labels = (predictions > 0.5).flatten()
    
    # Display images with predictions
    plt.figure(figsize=(20, 10))
    for i in range(num_images):
        plt.subplot(2, 5, i+1)
        plt.imshow(images[i])
        
        true_label = "Fake" if true_labels[i] == 1 else "Real"
        pred_label = "Fake" if pred_labels[i] == 1 else "Real"
        
        color = "green" if true_labels[i] == pred_labels[i] else "red"
        
        plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize some predictions
visualize_predictions(model, test_generator)

In [ ]:
# Calculate confusion matrix
test_generator.reset()
y_true = test_generator.classes
y_pred = (model.predict(test_generator) > 0.5).flatten().astype(int)

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Real', 'Fake'],
            yticklabels=['Real', 'Fake'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Real', 'Fake']))

# Cell 24: Save the model
# Save the model for future use
model.save('deepfake_detector_final.h5')
print("Model saved successfully!")

In [ ]:
#code is done